# Final project on topic
# Named entity recognition: recognition quality of ontological objects depending on the depth of hyponymy

### By Dochkina V. and Bragin D.

# General information

Named-entity recognition (NER) is one of the most important tasks in the field of natural language processing (Natural language
processing, NLP).

The task of NER is to highlight and classify named entities in the text. A named entity is a word or phrase meaning subject or phenomenon of a certain category. Named entities are personalities (PERSON), names of organizations (ORGANIZATION), locations (LOCATION) and others. Example of tagged text: 
"[ORGANIZATION Skoltech], is a Russian university, located in [LOCATION Moscow]"

Hyponyms are words that name objects (properties, attributes) as
elements of a class (set) and consisting in a relationship of hyponymy with a word - the name of this class (hyperonym). For example, the words: ring, bracelet, necklace are
hyponyms in relation to the word jewelry, and vice versa, from the point of view of the inverse relationship, jewelry acts as a hyperonym in relation to the words ring, bracelet,
necklace.


# Goal

The goal of our study is to highlight the object and then determine it to a specific type, that is, the comparison of a hyponym with its hyperonym. At the same time, the depth of hyponymy can be multilevel. For example, "Mont Blanc" refers to the Mountain class,
which in turn belongs to the Geographic location class.
The obtained information about ontological objects is useful in various tasks related to natural language processing, including answers to questions posed by the user and extracting dependencies between objects.

To do that texts in the English language are used, followed by extraction of named entities. In the problem, proposals are given in the form of a sequence of tokens $w =
= (w_1, w_2, ..., w_n)$ , and we need to output a sequence of tags $y = (y_1, y_2, ..., y_n)$.

For this, the BIO markup format is used and each ontological object receives a tag corresponding to a given level of hyponymy. For every entity is built
a chain of corresponding hyperonyms based on data from the WordNet system. Sets
data differ depending on the given depth of hyponymy. The problem is solved with
using standard NER models.

# Main methods

For the NER task, a neural network model with a hybrid architecture is used: BiLSTM-CRF.

### Recurrent Neural Networks

Unlike multilayer perceptrons, recurrent networks can use their internal memory to process sequences of arbitrary length. In real
regular RNN only stores information about a short context (gradient attenuation).
LSTM lacks such a drawback - a non-network recursive block consisting of the following elements: Main layers (as in usual RNN), three sigmoidal filter layers, Cell
memory. Formulas for these components:



\begin{equation}  
i_t= \sigma( W_{ix}x_t+ W_{ih}h_{t-1}+b_i),
\end{equation}
\begin{equation}   
f_t = \sigma( W_{fx}x_t+ W_{fh}h_{t-1}+b_f),
\end{equation}
 \begin{equation}  
c_n = g( W_{cx}x_t+ W_{ch}h_{t-1}+b_c),
\end{equation}
 \begin{equation}  
c_t = f_{tt-1}+i_t  \circ c_n,
\end{equation}
\begin{equation}  
h_t = o_t\circ g(c_t),
\end{equation}
\begin{equation}  o_t =  \sigma( W_{ox}x_t+ W_{oh}h_{t-1}+b_o)
\end{equation}

Variables:

$ x_t $  - input vector
$ h_ {t} $ is the output vector,
$ c_ {t} $ - state vector,
$ W, U $ and $ b $ are parameter matrices and a vector,
$ f_t, i_ {t} $ and $ {o_ {t}} $ are the gate vectors,
$ f_t $ - vector of forgetting gate, weight of remembering old information,
$ {i_ {t}} $ - vector of the input gate, weight of receiving new information,
$ {o_ {t}} $ - vector of the output gate, candidate for the output.


Activation functions:


$ {\sigma _{g}} $ : sigmoida based.

$ \sigma _{c} $ : hyperbolitc tangens based.

$ {\ sigma _ {h}} $: based on the hyperbolic tangent, but in the work on the eyes (inspection holes) for LSTM it is assumed that $ {\ sigma _ {h} (x) = x} $


**Sigmoid**:

$$f (x) = \frac {1} {1 + e ^ x}$$
- $ f (x) \ in [0,1] $ - allows modeling probabilities
 - Differentiable and monotonous
 - Generalized by the softmax function


**Hyperbolic tangens**:

$$f (x) = tanh (x) $$
- All properties of sigmoid
 - The value of $ f (x) $ is always non-negative
 - Commonly used for binary classification

### Bidirectional recurrent neural networks (Bi-LSTM) 
Bi-LSTM were designed to encode each element in a sequence, taking into account left and right contexts,
making it one of the best options for the NER task. Normal LSTM takes into account
only past contexts, bidirectional and future ones. The calculation of the bidirectional model consists of two stages: the first layer calculates the representation of the left context and the second layer calculates the representation of the right context. Outputs of these steps then
combine to obtain a complete representation of an input sequence element. Bidirectional LSTM encoders have been shown to be useful in many NLP tasks,
such as machine translation, answering questions, and especially to solve the NER problem.

### CRF model for NER

Conditional Random Field (CRF) is a probabilistic model for structural forecasting, which has been successfully applied in various fields, including natural language processing.
The CRF model is trained to predict the vector of $  {y} = {y_0, y_1, ... y_n} $ tags, taking into account the sentence $ {x} = {x_0, x_1, .., x_N}. $ For this, the conditional probability:
 
$$ p ( {y} \mid  {x}) =
\frac {e ^ {Score (\vec {x}, \vec {y})}} {\sum_ {y ^ {'}} e ^ {Score (\vec {x}, \vec {y})} } $$
where $ Score $ is calculated by the formula:
 
$$ Score (\vec {x}, \vec {y}) = \sum_ {i = 0} ^ {n} A_ {y_i, y_ {i + 1}} + \sum_ {i = 1} ^ {n } P_i y_i $$

where $ A_ {y_i, y_ {i + 1}} $ denotes the probability of transition from the tag $ i $ to the tag $ j $, $ P_ {i, j}$ is the probability of transition, which represents the estimate of the $j-$th tag of the ith the words.

### Combination of Bi-LSTM and CRF model

In a combined model, the characters of each word in a sentence are fed into the network
Bi-LSTM in order to capture the character features of words. Then these vector representations of the character level are combined with the word embedding vectors and
transferred to another Bi-LSTM network. This network computes a sequence of ratings that represent the tag probabilities for each word in a sentence. To raise
prediction accuracy, CRF level is trained to apply restrictions depending on
tag order. For example, in the BIO scheme tag (B - Begin, I - Inside, O - Other) I never
appears at the beginning of a sentence, or O I B O is an invalid tag sequence.

# Preparation 

**Downloading WordNet lexical database to determine synsets**

In [2]:
from nltk.corpus import wordnet as wn

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Даниил
[nltk_data]     Брагин\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [3]:
!pip install pptree

  Stored in directory: C:\Users\1859~1\AppData\Local\pip\Cache\wheels\68\b2\d4\6028fa5c48fc258dd13129ddb85dedc8e0ca85a11fdaff9829
Successfully built pptree


In [4]:
from pptree import *

Visualization of tree by using wordnet (with the help of the functionget_tree) for depth = 3

In [5]:
word='j'
def get_tree(word, tr=Node(word), first=1, hmax=3, current_h=1):
    if first == 1: 
        try:
            word_sn = wn.synsets(word)[0]
            nod = Node(word)
        except:
            print('There is no such word in the dictionary') 
            return Node('error')
    else:
        word_sn = word
        nod = Node(word_sn.lemma_names()[0], tr)
    
    current_h += 1
    
    if current_h <= hmax:
        
        nodes = word_sn.hyponyms()
        
        for node in nodes:
            _ = get_tree(node, nod, first=0, current_h=current_h, hmax=hmax)
        
    return nod

Visualization of tree for english word 'machine'

In [8]:
word = 'machine'
word_sn = wn.synsets(word)
print(word_sn[1].hyponyms())

t = get_tree(word, hmax=2)

print_tree(t) 

[Synset('cyborg.n.01')]
        ┌assembly
        ├bagger
        ├calculator
        ├calender
        ├cash_machine
        ├comber
        ├computer
        ├concrete_mixer
        ├corker
        ├cotton_gin
        ├decoder
        ├farm_machine
        ├franking_machine
        ├hop-picker
        ├machine_tool
        ├machinery
        ├milking_machine
        ├motor
        ├pavior
        ├perpetual_motion_machine
        ├pile_driver
 machine┤
        ├Zamboni
        ├workhorse
        ├trimmer
        ├time_machine
        ├textile_machine
        ├stapler
        ├staple_gun
        ├stamp
        ├sorter
        ├snow_thrower
        ├slot_machine
        ├slicer
        ├simulator
        ├self-feeder
        ├riveting_machine
        ├record_player
        ├printer
        ├press
        ├press
        ├power_tool
        └power_shovel


Here we are going to choose only some of the leaves of hyponymy/hypernomy tree

In [9]:
root = Node(name='__root')

first = {'physical entity': Node('physical entity', parent=root), 
         'abstract entity': Node('abstract entity', parent=root)}

second = {'physical object': Node('physical object', parent=first['physical entity']), 
          'physical process': Node('physical process', parent=first['physical entity']),
          
          'physiological feature': Node('physiological feature', parent=first['abstract entity']),
          'attribute': Node('attribute', parent=first['abstract entity']),
          'measure': Node('measure', parent=first['abstract entity'])}

third = {'geological formation': Node('geological formation', parent=second['physical object']),
         'land': Node('land', parent=second['physical object']),
         'location': Node('location', parent=second['physical object']),
         'living thing': Node('living thing', parent=second['physical object']),
         'natural object': Node('natural object', parent=second['physical object']),
         'artifact': Node('artifact', parent=second['physical object']),
         
         'economic process': Node('economic process', parent=second['physical process']),
         'human process': Node('human process', parent=second['physical process']),
         'industrial process': Node('industrial process', parent=second['physical process']),
         'natural process': Node('natural process', parent=second['physical process']),
         'organic process': Node('organic process', parent=second['physical process']),
          
         'cognition': Node('cognition', parent=second['physiological feature']),
         'motivation': Node('motivation', parent=second['physiological feature']),
         'event': Node('event', parent=second['physiological feature']),
         
         'time': Node('time', parent=second['attribute']),
         'property': Node('property', parent=second['attribute']),
         'quality': Node('quality', parent=second['attribute']),
         
         'value': Node('value', parent=second['measure']),
         'volume': Node('volume', parent=second['measure']),
         'time unit': Node('time unit', parent=second['measure']),
         'time interval': Node('time interval', parent=second['measure']),

         }

print_tree(root)

                                        ┌economic process
                                        ├human process
                       ┌physical process┤
                       │                ├organic process
                       │                ├natural process
                       │                └industrial process
       ┌physical entity┤
       │               │               ┌geological formation
       │               │               ├land
       │               │               ├location
       │               └physical object┤
       │                               ├artifact
       │                               ├natural object
       │                               └living thing
 __root┤
       │                                     ┌cognition
       │               ┌physiological feature┤
       │               │                     ├event
       │               │                     └motivation
       └abstract entity┤
                       │       ┌value
         

Function that is going to return hyponyms of the chosen word

In [10]:
def get_all_hyps_by_word(word, first=1):
    
    hypo = lambda s: s.hyponyms()
    words = []
    wsynsets = [x for x in wn.synsets(word) if x.pos()=='n']
    clos = lambda s: s.closure(hypo) 
    
    synsets = []
    for wsynset in wsynsets:
        synsets += clos(wsynset)
    for synset in synsets:
        words = words + synset.lemma_names()
        
    return list(set(words))   

In [11]:
def space_to__(a):
    return '_'.join(a.split())

def __to_space(a):
    return ' '.join(a.split('_'))

In [12]:
dictionary = set()

for key in third:
    node = third[key]
    print(space_to__(key))
    words = get_all_hyps_by_word(word=space_to__(key))
    for word in words:
        word = __to_space(word)
        
        if word not in dictionary:
            Node(word, parent=node)
        
        dictionary.add(word)
        

geological_formation
land
location
living_thing
natural_object
artifact
economic_process
human_process
industrial_process
natural_process
organic_process
cognition
motivation
event
time
property
quality
value
volume
time_unit
time_interval


Amount of children nodes for a certain hyponym

In [0]:
print(len(list(dictionary)))
for key in third:
    node = third[key]
    print(key, len(node.children))

65082
geological formation 271
land 199
location 1721
living thing 30042
natural object 1211
artifact 13431
economic process 22
human process 78
industrial process 13
natural process 410
organic process 800
cognition 5138
motivation 69
event 8512
time 168
property 1492
quality 1418
value 2
volume 0
time unit 49
time interval 36


In [0]:
hypo = lambda s: s.hyponyms()
list(wn.synsets('time')[1].closure(hypo))

[]

In [0]:
get_all_hyps_by_word('time')[:10]

['gloam',
 'Pacific_Standard_Time',
 'nightfall',
 'Last_Judgment',
 'June_21',
 'upbeat',
 'allegro',
 'good_old_days',
 'twelve_noon',
 'Atlantic_Standard_Time']

# Model

In [0]:
!pip install deeppavlov

     |████████████████████████████████| 737kB 3.3MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 25.2MB 96kB/s 
     |████████████████████████████████| 8.0MB 14.5MB/s 
     |████████████████████████████████| 2.1MB 25.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 163kB 57.7MB/s 
     |████████████████████████████████| 6.7MB 50.8MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 10.1MB 47.6MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 317kB 57.9MB/s 
     |████████████████████████████████| 2.8MB 46.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 17.3MB 102kB/s 
     |████████████████████████████████| 7.1MB 24.9MB/s 
     |████████████████████████████████| 5.1MB 40.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     

In [0]:
!python -m deeppavlov install ner_ontonotes

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:12:25.807 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_ontonotes' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_ontonotes.json'
     |████████████████████████████████| 24.2MB 108kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 109.2MB 21kB/s 
     |████████████████████████████████| 3.2MB 19.7MB/s 
     |████████████████████████████████| 491kB 50.0MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15

Setting up 3 levels of depht

In [0]:
levels = [['entity']]
levels.append(['physical entity', 'abstract entity'])
levels.append(['physical object', 'physical process', 'psychological feature', 'attribute', 'measure'])
levels.append(['geological formation', 'land', 'location', 'living thing', 'natural object', 'artifact'] +
        ['economic process', 'human process', 'industrial process', 'natural process', 'organic process'] +
        ['cognition', 'motivation', 'event', 'time', 'property', 'quality'] +
        ['value', 'volume', 'time unit', 'time interval'])

In [0]:
import random

In [0]:
class Node:
    def __init__(self, synset, parent=None):
        self.children = []
        if parent:
            parent.children.append(self)
        self.parent = parent
        self.synset = synset
       
    def __str__(self):
        return ', '.join(self.synset.lemma_names() )#[0]
    

# nodes instead of synset store their parents name in dict,
# like names of all of its childrens, childrens of children etc.
class Leave:     
    def __init__(self, tag_dict, parent):
        self.children = []
        parent.children = [self]
        self.parent = parent
        self.dict = tag_dict
        
    def __str__(self):
        return '**leave: ' + str(len(self.dict))#', '.join(list(self.dict))
    
def hypo_closure(synset):
    return synset.closure(hypo)

def add_child(parent, synset):
    child = Node(synset, parent)
    return child

def add_dict_child(parent):           
    # filling up dict
    hypo_clsr = list(hypo_closure(parent.synset))
    child_names = []
    
    for hyponym in hypo_clsr:
        child_names += hyponym.lemma_names()
        
    #adding the name of the parent itself
    # dict holds all words corresponding to tag-parent
    child_names += parent.synset.lemma_names()
    tag_dict = set(child_names)
    child = Leave(tag_dict, parent)
        
    return child

In [0]:
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()

#Hierarchy class
class Hierarchy:
    
    def __init__(self, levels, root_synset):
        self.levels = levels
        self.leaves = []
        self.max_depth = len(levels) - 1
        self.root_synset = root_synset
        self.root = Node(root_synset)

    def build(self, parent=None, depth=1) :
        if not parent :
            parent = self.root
        
        if depth > self.max_depth:
            child = add_dict_child(parent)
            self.leaves.append(child)
            return

        for synset in hypo(parent.synset):
            #print (hypo(parent.synset))
            if set(self.levels[depth]) & set([x.replace('_', ' ') for x in synset.lemma_names()]):
                child = add_child(parent, synset)
                self.build(child, depth+1)

        return parent
    
    def get_tag(self, string):
        string = string.replace(' ', '_')
        candidates = []
        for leave in self.leaves:
            if string in leave.dict:
                candidates.append(leave.parent.synset.lemma_names()[0])
            
        return random.choice(candidates) if candidates else 'O'
      
    def decrease_level(self):
        new_leaves = []
        new_tags = set()
        for leave in self.leaves:
            new_tags.add(leave.parent.parent)
    
        for new_tag in new_tags:
            new_tag_dict = set()
            for new_tag_child in new_tag.children:
                new_tag_dict |= new_tag_child.children[0].dict # tag has just one  child
            
            new_tag_dict |= set(new_tag.synset.lemma_names())          
            new_leaves.append(Leave(new_tag_dict, new_tag))            
            
        self.max_depth -= 1
        self.leaves = new_leaves
    

In [0]:
hierarchy = Hierarchy(levels, wn.synsets('entity')[0])
entity = hierarchy.build()

#whole = wn.synset('whole.n.02')
#malually add artifact, living_thing, synset
artifact = wn.synset('artifact.n.01')
living_thing = wn.synset('living_thing.n.01')
natural_object = wn.synset('natural_object.n.01')


for i in entity.children :
    if i.__str__() == 'physical_entity':
        for j in i.children :
            if j.__str__() == 'object, physical_object':
                for synset in [artifact, living_thing, natural_object]:
                    child = add_child(j, synset)
                    hierarchy.leaves.append(add_dict_child(child))
for i in entity.children :
    if i.__str__() == 'abstraction, abstract_entity':
        for j in i.children :
            if j.__str__() == 'measure, quantity, amount':
                    synset = wn.synset('time_period.n.01')
                    child = add_child(j, synset)
                    hierarchy.leaves.append(add_dict_child(child))
                    



Just some experiments

In [0]:
[hierarchy.get_tag('dog') for _ in range(7)]

['living_thing',
 'artifact',
 'artifact',
 'living_thing',
 'living_thing',
 'artifact',
 'artifact']

In [0]:
hierarchy.get_tag('hedgehog')

'living_thing'

In [0]:
hierarchy.get_tag('year')

'time_period'

In [0]:
hierarchy.get_tag('pure mathematics')

'cognition'

In [0]:
#hierarchy.decrease_level()
#print_tree(hierarchy.root)

In [0]:
hierarchy.get_tag('pure mathematics')

'cognition'

In [0]:
#hierarchy.decrease_level()
#hierarchy.get_tag('pure mathematics')

In [0]:
!wget https://github.com/Intelligent-Systems-Phystech/2018-Project-19/raw/master/X.txt

--2019-12-18 16:13:42--  https://github.com/Intelligent-Systems-Phystech/2018-Project-19/raw/master/X.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Intelligent-Systems-Phystech/2018-Project-19/master/X.txt [following]
--2019-12-18 16:13:43--  https://raw.githubusercontent.com/Intelligent-Systems-Phystech/2018-Project-19/master/X.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7965488 (7.6M) [text/plain]
Saving to: ‘X.txt’

X.txt               100%[===================>]   7.60M  --.-KB/s    in 0.1s    

2019-12-18 16:13:43 (79.0 MB/s) - ‘X.txt’ saved [7965488/7965488]



In [0]:
#Path to the dataset - WOS5736
with open('X.txt') as f:
    content = f.readlines()

In [0]:
import re
len(content)
texts = []
for one_text in content:
    texts.append(one_text.lower())
text = texts[0]
texts_array = []
for text in texts:
    texts_array.append(re.findall(r"[\w']+|[.,!?;\'\"]", text))

In [0]:
def tag_text(hierarchy, text):
    string = ''
    r = 0
    for word_number in range(0,len(text)-1):
        if r == 1: 
            r = 0
            continue
        if text[word_number]=='.':
            string += '. O\n\n'
        else:
            if hierarchy.get_tag(text[word_number]+' '+text[word_number+1]) == 'O':
                if hierarchy.get_tag(text[word_number]) == 'O':
                    string += text[word_number] + ' O\n'
                else:
                    string += text[word_number] + ' B-' + hierarchy.get_tag(text[word_number]) + '\n'
            else:
                string += text[word_number] + ' B-' + hierarchy.get_tag(text[word_number]) + '\n'
                string += text[word_number+1] + ' I-' + hierarchy.get_tag(text[word_number]) + '\n'
                r = 1
    string += '. O\n\n'
    return string


In [0]:
markup_third_lvl = []
for text in texts_array[:5]: 
    markup_third_lvl.append(tag_text(hierarchy, text))

In [0]:
#print(markup_third_lvl[0])


In [0]:
markup_second_lvl = []
hierarchy.decrease_level()
for text in texts_array[:5]:
    markup_second_lvl.append(tag_text(hierarchy, text))

In [0]:
markup_first_lvl = []
hierarchy.decrease_level()
for text in texts_array[:5]:
    markup_first_lvl.append(tag_text(hierarchy, text))

In [0]:
def splitter(markup):
    return markup[:3], markup[3:4], markup[4:]

In [0]:
markups = [markup_first_lvl, markup_second_lvl, markup_third_lvl]

for i, markup in enumerate(markups):
    train, test, validation = splitter(markup)
    
    import os

    dest_dir = os.path.join("dataset_" + str(i+1))
    try:
        os.makedirs(dest_dir)
    except OSError:
        pass # already exists
    
    with open("dataset_" + str(i+1) +"/train.txt", "w") as file:
        for j in train:
            print(j, file=file)
    with open("dataset_" + str(i+1)+ "/test.txt", "w") as file:
        for j in test:
            print(j, file=file)
    with open("dataset_" + str(i+1) + "/validation.txt", "w") as file:
        for j in validation:
            print(j, file=file)

In [0]:
!python -m deeppavlov install ner_ontonotes

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:13:46.255 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_ontonotes' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_ontonotes.json'


In [0]:
!python -m deeppavlov download ner_ontonotes

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:13:52.691 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_ontonotes' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_ontonotes.json'
2019-12-18 16:13:53.484 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_v3_cpu_compatible.tar.gz?config=ner_ontonotes to /root/.deeppavlov/ner_ontonotes_v3_cpu_compatible.tar.gz
100% 8.13M/8.13M [00:02<00:00, 4.00MB/s]
2019-12-18 16:13:55.520 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/ner_ontonotes_v3_cpu_compatible.tar.gz archive into /root/.deeppavlov/models
2019-12-18 16:13:56.226 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt?config=ner_ontonotes to /root/.deeppavlov/d

In [0]:
from deeppavlov import configs, train_model

In [0]:
#!wget https://raw.githubusercontent.com/deepmipt/DeepPavlov/0.1.0/deeppavlov/configs/ner/ner_ontonotes.json

In [0]:
# import json

# with open('my_config.json', 'r+') as f:
#     data = json.load(f)
#     data['dataset_reader']['data_path'] = "/dataset_1/"
#     f.seek(0)       
#     json.dump(data, f, indent=4)
#     f.truncate()     
    
# data = json.load(open('my_config.json', 'r+')) 

# data['dataset_reader']['data_path'] = "/dataset_1/"



In [0]:
from deeppavlov import train_model, configs
# ner_model = train_model(data)

In [0]:
#ner_few = json.load(open('ner_few.json', 'r+'))
#ner_model = train_model(ner_few, download = True)

In [0]:
#asda = json.load(open('ner_conll2003_pos.json', 'r+'))
#ner_model = train_model(configs.ner.ner_conll2003_pos, download = True)

In [0]:
hierarchy.get_tag('term')

'physical_entity'

In [0]:
!python -m deeppavlov install ner_conll2003

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:14:52.314 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'


In [0]:
config = {
    "dataset_reader": {
        "class_name": "conll2003_reader",
        "data_path": "./onthology_objects_data_new/dataset_1/",
        "dataset_name": "conll2003",
        "provide_pos": False
    },
    "dataset_iterator": {
        "class_name": "data_learning_iterator",
        "seed": 42
    },
    "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "in": ["x"],
        "class_name": "lazy_tokenizer",
        "out": ["x_tokens"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "str_lower",
        "out": ["x_lower"]
      },
      {
        "in": ["x_lower"],
        "class_name": "sanitizer",
        "nums": True,
        "out": ["x_san"]
      },
      {
        "in": ["x_san"],
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "special_tokens": ["<UNK>"],
        "fit_on": ["x_san"],
        "save_path": "{MODELS_PATH}/onthology_objects/word.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/word.dict",
        "out": ["x_tok_ind"]
      },
      {
        "in": ["y"],
        "id": "tag_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["y"],
        "save_path": "{MODELS_PATH}/onthology_objects/tag.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/tag.dict",
        "out": ["y_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "char_splitter",
        "out": ["x_char"]
      },
      {
        "in": ["x_char"],
        "id": "char_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["x_char"],
        "save_path": "{MODELS_PATH}/onthology_objects/char.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/char.dict",
        "out": ["x_char_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "mask",
        "out": ["mask"]
      },
      {
        "in": ["x_san"],
        "id": "glove_emb",
        "class_name": "glove",
        "pad_zero": True,
        "load_path": "{DOWNLOADS_PATH}/embeddings/glove.6B.100d.txt",
        "out": ["x_emb"]
      },
      {
        "id": "embeddings",
        "class_name": "emb_mat_assembler",
        "embedder": "#glove_emb",
        "vocab": "#word_vocab"
      },
      {
        "id": "embeddings_char",
        "class_name": "emb_mat_assembler",
        "character_level": True,
        "emb_dim": 32,
        "embedder": "#glove_emb",
        "vocab": "#char_vocab"
      },
      {
        "id": "capitalization",
        "class_name": "capitalization_featurizer",
        "in": ["x_tokens"],
        "out": ["cap"]
      },
      {
        "in": ["x_emb", "mask", "x_char_ind", "cap"],
        "in_y": ["y_ind"],
        "out": ["y_predicted"],
        "class_name": "ner",
        "main": True,
        "token_emb_dim": "#glove_emb.dim",
        "n_hidden_list": [128],
        "net_type": "rnn",
        "cell_type": "lstm",
        "use_cudnn_rnn": True,
        "n_tags": "#tag_vocab.len",
        "capitalization_dim": "#capitalization.dim",
        "char_emb_dim": "#embeddings_char.dim",
        "save_path": "{MODELS_PATH}/onthology_objects/model_no_pos",
        "load_path": "{MODELS_PATH}/onthology_objects/model_no_pos",
        "char_emb_mat": "#embeddings_char.emb_mat",
        "two_dense_on_top": True,
        "use_crf": True,
        "use_batch_norm": True,
        "embeddings_dropout": True,
        "top_dropout": True,
        "intra_layer_dropout": True,
        "l2_reg": 0,
        "learning_rate": 1e-2,
        "dropout_keep_prob": 0.5
      },
      {
        "ref": "tag_vocab",
        "in": ["y_predicted"],
        "out": ["tags"]
      }
    ],
    "out": ["x_tokens", "tags"]
  },
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "metrics": [
            {
                "name": "ner_f1",
                "inputs": [
                    "y",
                    "tags"
                ]
            }
        ],
        "validation_patience": 7,
        "val_every_n_epochs": 1,
        "log_every_n_epochs": -1,
        "show_examples": False,
        "validate_best": True,
        "test_best": True
    },
    "metadata": {
        "variables": {
            "ROOT_PATH": ".",
            "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
            "MODELS_PATH": "{ROOT_PATH}/models"
        },
        "requirements": [
            "{DEEPPAVLOV_PATH}/requirements/gensim.txt",
            "{DEEPPAVLOV_PATH}/requirements/tf.txt"
        ],
        "labels": {
            "telegram_utils": "NERCoNLL2003Model",
            "server_utils": "NER"
        },
        "download": [
            {
                "url": "http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
                "subdir": "{DOWNLOADS_PATH}/embeddings"
            }
        ]
    }
}

In [0]:
import json

with open("config.json", "w") as fout:
  json.dump(config, fout, ensure_ascii=False, indent=2)

In [0]:
!pip install deeppavlov
!python -m deeppavlov download config.json
!python -m deeppavlov install ner_conll2003


email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:15:02.357 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt?config=config to /content/downloads/embeddings/glove.6B.100d.txt
347MB [00:27, 12.6MB/s]
email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:15:31.628 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'


In [0]:
!wget http://files.deeppavlov.ai/datasets/onthology_objects_data_new.zip
! unzip onthology_objects_data_new.zip


--2019-12-18 16:15:37--  http://files.deeppavlov.ai/datasets/onthology_objects_data_new.zip
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346661 (339K) [application/zip]
Saving to: ‘onthology_objects_data_new.zip’

onthology_objects_d 100%[===================>] 338.54K   578KB/s    in 0.6s    

2019-12-18 16:15:38 (578 KB/s) - ‘onthology_objects_data_new.zip’ saved [346661/346661]

Archive:  onthology_objects_data_new.zip
   creating: onthology_objects_data_new/
   creating: onthology_objects_data_new/dataset_1/
  inflating: onthology_objects_data_new/dataset_1/test.txt  
  inflating: onthology_objects_data_new/dataset_1/train.txt  
  inflating: onthology_objects_data_new/dataset_1/valid.txt  
   creating: onthology_objects_data_new/dataset_2/
  inflating: onthology_objects_data_new/dataset_2/test.txt  
  inflating: ontholog

In [0]:
from deeppavlov import build_model, train_model
model = train_model(config)

2019-12-18 16:15:40.84 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2019-12-18 16:15:41.99 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/models/onthology_objects/word.dict]
2019-12-18 16:15:41.122 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/models/onthology_objects/tag.dict]









Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Instructions for updating:
Use `tf.cast` instead.


2019-12-18 16:16:23.956 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


...

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 75.5578}, "time_spent": "0:00:01"}}


2019-12-18 16:23:44.962 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 4290 tokens with 969 phrases; found: 936 phrases; correct: 0.

precision:  74.25%; recall:  71.72%; FB1:  72.97

	O: precision:  81.82%; recall:  52.94%; F1:  64.29 22

	abstraction: precision:  75.08%; recall:  80.00%; F1:  77.46 618

	physical_entity: precision:  71.96%; recall:  60.00%; F1:  65.44 296




{"test": {"eval_examples_count": 222, "metrics": {"ner_f1": 72.9659}, "time_spent": "0:00:01"}}


2019-12-18 16:23:45.227 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/models/onthology_objects/word.dict]
2019-12-18 16:23:45.238 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/models/onthology_objects/tag.dict]
2019-12-18 16:23:45.240 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/models/onthology_objects/char.dict]
2019-12-18 16:23:45.241 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/downloads/embeddings/glove.6B.100d.txt`]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration not

INFO:tensorflow:Restoring parameters from /content/models/onthology_objects/model_no_pos


In [0]:
test_str = 'Wikipedia is a free online encyclopedia, created and edited by volunteers around the world and hosted by the Wikimedia Foundation.'
res = model.compute([test_str])
res = [res[0][0], res[1][0]]
readable = []
for i in range(len(res[0])):
  readable.append([res[0][i], res[1][i]])
readable

[['Wikipedia', 'O'],
 ['is', 'O'],
 ['a', 'O'],
 ['free', 'O'],
 ['online', 'O'],
 ['encyclopedia', 'B-abstraction'],
 [',', 'O'],
 ['created', 'O'],
 ['and', 'O'],
 ['edited', 'O'],
 ['by', 'O'],
 ['volunteers', 'O'],
 ['around', 'O'],
 ['the', 'O'],
 ['world', 'B-abstraction'],
 ['and', 'O'],
 ['hosted', 'O'],
 ['by', 'O'],
 ['the', 'O'],
 ['Wikimedia', 'O'],
 ['Foundation', 'B-abstraction'],
 ['.', 'O']]

In [0]:
tag_text(hierarchy, re.findall(r"[\w']+|[.,!?;\'\"]", test_str)).split('\n')

['Wikipedia O',
 'is O',
 'a O',
 'free O',
 'online O',
 'encyclopedia B-physical_entity',
 ', O',
 'created O',
 'and O',
 'edited O',
 'by O',
 'volunteers O',
 'around O',
 'the O',
 'world B-abstraction',
 'and O',
 'hosted O',
 'by O',
 'the O',
 'Wikimedia O',
 'Foundation O',
 '. O',
 '',
 '']

In [0]:
hierarchy.get_tag('volunteer')

'physical_entity'

In [0]:
wn.morphy('volunteers')

'volunteer'

** GLUBINA 2 **

In [0]:
!pip install deeppavlov

In [0]:
!python -m deeppavlov install ner_conll2003

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:24:32.860 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'


In [0]:
!python -m deeppavlov install intents_dstc2_big

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:24:39.470 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'intents_dstc2_big' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/classifiers/intents_dstc2_big.json'
     |████████████████████████████████| 153kB 3.5MB/s 
  Cloning https://github.com/deepmipt/fastText.git to /tmp/pip-install-uyycg_x4/fastText
  Running command git clone -q https://github.com/deepmipt/fastText.git /tmp/pip-install-uyycg_x4/fastText
  Created wheel for fastText: filename=fasttext-0.8.22-cp36-cp36m-linux_x86_64.whl size=2138880 sha256=b1f74e1adab46b2474657b8da7bbaf82f539d0c70f116621fa05eea68e947a65
  Stored in directory: /tmp/pip-ephem-wheel-cache-0rhuxvcg/wheels/19/a8/8d/f2d5f95a573a8bbf7e818c1914e80d72899233bfb111c04539
Successfully built fastText


In [0]:
config = {
    "dataset_reader": {
        "class_name": "conll2003_reader",
        "data_path": "./onthology_objects_data_new/dataset_2/",
        "dataset_name": "conll2003",
        "provide_pos": False
    },
    "dataset_iterator": {
        "class_name": "data_learning_iterator",
        "seed": 42
    },
    "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "in": ["x"],
        "class_name": "lazy_tokenizer",
        "out": ["x_tokens"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "str_lower",
        "out": ["x_lower"]
      },
      {
        "in": ["x_lower"],
        "class_name": "sanitizer",
        "nums": True,
        "out": ["x_san"]
      },
      {
        "in": ["x_san"],
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "special_tokens": ["<UNK>"],
        "fit_on": ["x_san"],
        "save_path": "{MODELS_PATH}/onthology_objects/word_2_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/word_2_ft.dict",
        "out": ["x_tok_ind"]
      },
      {
        "in": ["y"],
        "id": "tag_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["y"],
        "save_path": "{MODELS_PATH}/onthology_objects/tag_2_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/tag_2_ft.dict",
        "out": ["y_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "char_splitter",
        "out": ["x_char"]
      },
      {
        "in": ["x_char"],
        "id": "char_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["x_char"],
        "save_path": "{MODELS_PATH}/onthology_objects/char_2_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/char_2_ft.dict",
        "out": ["x_char_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "mask",
        "out": ["mask"]
      },
      {
        "in": ["x_san"],
        "id": "glove_emb",
        "class_name": "fasttext",
        "pad_zero": True,
        "dim": 300,
        "load_path": "{DOWNLOADS_PATH}/embeddings/wiki.en.bin",
        "out": ["x_emb"]
      },
      {
        "id": "embeddings",
        "class_name": "emb_mat_assembler",
        "embedder": "#glove_emb",
        "vocab": "#word_vocab"
      },
      {
        "id": "embeddings_char",
        "class_name": "emb_mat_assembler",
        "character_level": True,
        "emb_dim": 32,
        "embedder": "#glove_emb",
        "vocab": "#char_vocab"
      },
      {
        "id": "capitalization",
        "class_name": "capitalization_featurizer",
        "in": ["x_tokens"],
        "out": ["cap"]
      },
      {
        "in": ["x_emb", "mask", "x_char_ind", "cap"],
        "in_y": ["y_ind"],
        "out": ["y_predicted"],
        "class_name": "ner",
        "main": True,
        "token_emb_dim": "#glove_emb.dim",
        "n_hidden_list": [128],
        "net_type": "rnn",
        "cell_type": "lstm",
        "use_cudnn_rnn": True,
        "n_tags": "#tag_vocab.len",
        "capitalization_dim": "#capitalization.dim",
        "char_emb_dim": "#embeddings_char.dim",
        "save_path": "{MODELS_PATH}/onthology_objects/model_no_pos_2_ft",
        "load_path": "{MODELS_PATH}/onthology_objects/model_no_pos_2_ft",
        "char_emb_mat": "#embeddings_char.emb_mat",
        "two_dense_on_top": True,
        "use_crf": True,
        "use_batch_norm": True,
        "embeddings_dropout": True,
        "top_dropout": True,
        "intra_layer_dropout": True,
        "l2_reg": 0,
        "learning_rate": 1e-2,
        "dropout_keep_prob": 0.5
      },
      {
        "ref": "tag_vocab",
        "in": ["y_predicted"],
        "out": ["tags"]
      }
    ],
    "out": ["x_tokens", "tags"]
  },
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "metrics": [
            {
                "name": "ner_f1",
                "inputs": [
                    "y",
                    "tags"
                ]
            }
        ],
        "validation_patience": 7,
        "val_every_n_epochs": 1,
        "log_every_n_epochs": -1,
        "show_examples": False,
        "validate_best": True,
        "test_best": True
    },
    "metadata": {
        "variables": {
            "ROOT_PATH": "~/.deeppavlov",
            "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
            "MODELS_PATH": "{ROOT_PATH}/models"
        },
        "requirements": [
            "{DEEPPAVLOV_PATH}/requirements/fasttext.txt",
            "{DEEPPAVLOV_PATH}/requirements/tf.txt"
        ],
        "labels": {
            "telegram_utils": "NERCoNLL2003Model",
            "server_utils": "NER"
        },
        "download": [
            {
                "url": "http://files.deeppavlov.ai/deeppavlov_data/embeddings/wiki.en.bin",
                "subdir": "{DOWNLOADS_PATH}/embeddings"
            }
        ]
    }
}

In [0]:
import json

with open("config.json", "w") as fout:
  json.dump(config, fout, ensure_ascii=False, indent=2)

In [0]:
! rm ./downloads/embeddings/wiki.en.bin

rm: cannot remove './downloads/embeddings/wiki.en.bin': No such file or directory


In [0]:
!python -m deeppavlov download config.json

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:25:22.595 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/embeddings/wiki.en.bin?config=config to /root/.deeppavlov/downloads/embeddings/wiki.en.bin
100% 8.49G/8.49G [23:09<00:00, 6.11MB/s]


In [0]:
model1 = train_model(config)

2019-12-18 16:48:32.378 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
2019-12-18 16:48:32.667 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/word_2_ft.dict]
2019-12-18 16:48:32.690 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/tag_2_ft.dict]
2019-12-18 16:48:34.66 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/char_2_ft.dict]
2019-12-18 16:48:34.75 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 16:48:50.209 INFO in 'deeppavlov.core.layers.tf_layers'['tf_la

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 3.5801}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 7}}


2019-12-18 16:49:04.432 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1028 phrases; found: 432 phrases; correct: 0.

precision:  45.14%; recall:  18.97%; FB1:  26.71

	O: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	attribute: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	measure: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	object: precision:  100.00%; recall:  2.02%; F1:  3.95 7

	process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	psychological_feature: precision:  44.24%; recall:  46.19%; F1:  45.19 425


2019-12-18 16:49:04.434 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best ner_f1 of 26.7123
2019-12-18 16:49:04.434 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2019-12-18 16:49:04.435 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /root/.deeppavlov/models/onthology_objects/model_no_pos_2_ft]


{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 26.7123}, "time_spent": "0:00:13", "epochs_done": 1, "batches_seen": 28, "train_examples_seen": 1770, "impatience": 0, "patience_limit": 7}}


2019-12-18 16:49:16.434 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1028 phrases; found: 833 phrases; correct: 0.

precision:  51.98%; recall:  42.12%; FB1:  46.53

	O: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	attribute: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	measure: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	object: precision:  62.16%; recall:  39.77%; F1:  48.51 222

	process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	psychological_feature: precision:  48.36%; recall:  72.48%; F1:  58.01 610


2019-12-18 16:49:16.436 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best ner_f1 of 46.5341
2019-12-18 16:49:16.436 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2019-12-18 16:49:16.437 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /root/.deeppavlov/models/onthology_objects/model_no_pos_2_ft]


...

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 68.5799}, "time_spent": "0:00:01"}}


2019-12-18 16:53:09.562 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 4290 tokens with 971 phrases; found: 968 phrases; correct: 0.

precision:  66.53%; recall:  66.32%; FB1:  66.43

	O: precision:  69.23%; recall:  52.94%; F1:  60.00 26

	attribute: precision:  46.94%; recall:  45.54%; F1:  46.23 98

	measure: precision:  71.88%; recall:  63.89%; F1:  67.65 32

	object: precision:  69.35%; recall:  75.42%; F1:  72.26 323

	process: precision:  47.22%; recall:  36.17%; F1:  40.96 36

	psychological_feature: precision:  69.76%; recall:  69.30%; F1:  69.53 453




{"test": {"eval_examples_count": 222, "metrics": {"ner_f1": 66.426}, "time_spent": "0:00:01"}}


2019-12-18 16:53:10.2 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/word_2_ft.dict]
2019-12-18 16:53:10.12 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/tag_2_ft.dict]
2019-12-18 16:53:10.13 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/char_2_ft.dict]
2019-12-18 16:53:10.15 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 16:53:20.944 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.co

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/onthology_objects/model_no_pos_2_ft


In [0]:
!python -m deeppavlov evaluate config.json

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-18 16:53:24.544 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2019-12-18 16:53:25.207 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/word_2_ft.dict]

In [0]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

config["train"]["show_examples"] = True
model1 = train_evaluate_model_from_config(config, to_train=False)

2019-12-18 16:53:52.553 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
2019-12-18 16:53:52.755 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/word_2_ft.dict]
2019-12-18 16:53:52.765 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/tag_2_ft.dict]
2019-12-18 16:53:52.767 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/char_2_ft.dict]
2019-12-18 16:53:52.768 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 16:54:06.496 INFO in 'deeppavlov.core.layers.tf_lay

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/onthology_objects/model_no_pos_2_ft


2019-12-18 16:54:10.78 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1028 phrases; found: 993 phrases; correct: 0.

precision:  69.79%; recall:  67.41%; FB1:  68.58

	O: precision:  100.00%; recall:  80.00%; F1:  88.89 16

	attribute: precision:  58.70%; recall:  52.60%; F1:  55.48 138

	measure: precision:  61.90%; recall:  48.15%; F1:  54.17 21

	object: precision:  73.93%; recall:  69.45%; F1:  71.62 326

	process: precision:  53.12%; recall:  69.86%; F1:  60.36 96

	psychological_feature: precision:  73.48%; recall:  71.50%; F1:  72.48 396




{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 68.5799}, "time_spent": "0:00:01", "examples": [{"x": ["the", "key", "consideration", "is", "whether", "and", "to", "what", "extent", "the", "administrative", "record", "data", "re", "examined", "retroactively", "a", "decade", "later", "for", "the", "original", "study's", "time", "period", "would", "yield", "comparable", "results", "to", "those", "based", "on", "data", "acquired", "at", "the", "time", "of", "the", "study", "."], "y_predicted": [["O", "B-object", "B-psychological_feature", "O", "O", "O", "O", "O", "B-attribute", "O", "B-object", "B-psychological_feature", "O", "O", "O", "O", "O", "B-measure", "O", "O", "O", "B-psychological_feature", "O", "B-psychological_feature", "B-measure", "O", "B-object", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-measure", "O", "O", "B-object", "O"], ["the", "key", "consideration", "is", "whether", "and", "to", "what", "extent", "the", "administrative", "record", "data", "re",

2019-12-18 16:54:10.795 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 4290 tokens with 971 phrases; found: 968 phrases; correct: 0.

precision:  66.53%; recall:  66.32%; FB1:  66.43

	O: precision:  69.23%; recall:  52.94%; F1:  60.00 26

	attribute: precision:  46.94%; recall:  45.54%; F1:  46.23 98

	measure: precision:  71.88%; recall:  63.89%; F1:  67.65 32

	object: precision:  69.35%; recall:  75.42%; F1:  72.26 323

	process: precision:  47.22%; recall:  36.17%; F1:  40.96 36

	psychological_feature: precision:  69.76%; recall:  69.30%; F1:  69.53 453




{"test": {"eval_examples_count": 222, "metrics": {"ner_f1": 66.426}, "time_spent": "0:00:01", "examples": [{"x": ["ber", "ep4", "and", "cd44v6", "were", "shown", "to", "be", "great", "markers", "for", "detecting", "lnmm", "."], "y_predicted": [["O", "O", "O", "O", "O", "O", "O", "O", "B-object", "O", "O", "O", "O", "O"], ["ber", "ep4", "and", "cd44v6", "were", "shown", "to", "be", "great", "markers", "for", "detecting", "lnmm", "."]], "y_true": ["O", "O", "O", "O", "O", "O", "O", "O", "B-object", "O", "O", "B-psychological_feature", "O", "O"]}, {"x": ["aims", "human", "papillomavirus", "hpv", "is", "known", "as", "causative", "for", "squamous", "cell", "carcinoma", "scc", "of", "the", "oropharynx", ",", "but", "is", "also", "found", "not", "infrequently", "in", "carcinomas", "of", "the", "sinonasal", "tract", "."], "y_predicted": [["O", "B-object", "O", "O", "O", "O", "O", "O", "O", "B-O", "I-O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ob

**GLUBINA 3**

In [0]:
config = {
    "dataset_reader": {
        "class_name": "conll2003_reader",
        "data_path": "./onthology_objects_data_new/dataset_3/",
        "dataset_name": "conll2003",
        "provide_pos": False
    },
    "dataset_iterator": {
        "class_name": "data_learning_iterator",
        "seed": 42
    },
    "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "pipe": [
      {
        "in": ["x"],
        "class_name": "lazy_tokenizer",
        "out": ["x_tokens"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "str_lower",
        "out": ["x_lower"]
      },
      {
        "in": ["x_lower"],
        "class_name": "sanitizer",
        "nums": True,
        "out": ["x_san"]
      },
      {
        "in": ["x_san"],
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "special_tokens": ["<UNK>"],
        "fit_on": ["x_san"],
        "save_path": "{MODELS_PATH}/onthology_objects/word_3_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/word_3_ft.dict",
        "out": ["x_tok_ind"]
      },
      {
        "in": ["y"],
        "id": "tag_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["y"],
        "save_path": "{MODELS_PATH}/onthology_objects/tag_3_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/tag_3_ft.dict",
        "out": ["y_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "char_splitter",
        "out": ["x_char"]
      },
      {
        "in": ["x_char"],
        "id": "char_vocab",
        "class_name": "simple_vocab",
        "pad_with_zeros": True,
        "fit_on": ["x_char"],
        "save_path": "{MODELS_PATH}/onthology_objects/char_3_ft.dict",
        "load_path": "{MODELS_PATH}/onthology_objects/char_3_ft.dict",
        "out": ["x_char_ind"]
      },
      {
        "in": ["x_tokens"],
        "class_name": "mask",
        "out": ["mask"]
      },
      {
        "in": ["x_san"],
        "id": "glove_emb",
        "class_name": "fasttext",
        "pad_zero": True,
        "dim": 300,
        "load_path": "{DOWNLOADS_PATH}/embeddings/wiki.en.bin",
        "out": ["x_emb"]
      },
      {
        "id": "embeddings",
        "class_name": "emb_mat_assembler",
        "embedder": "#glove_emb",
        "vocab": "#word_vocab"
      },
      {
        "id": "embeddings_char",
        "class_name": "emb_mat_assembler",
        "character_level": True,
        "emb_dim": 32,
        "embedder": "#glove_emb",
        "vocab": "#char_vocab"
      },
      {
        "id": "capitalization",
        "class_name": "capitalization_featurizer",
        "in": ["x_tokens"],
        "out": ["cap"]
      },
      {
        "in": ["x_emb", "mask", "x_char_ind", "cap"],
        "in_y": ["y_ind"],
        "out": ["y_predicted"],
        "class_name": "ner",
        "main": True,
        "token_emb_dim": "#glove_emb.dim",
        "n_hidden_list": [128],
        "net_type": "rnn",
        "cell_type": "lstm",
        "use_cudnn_rnn": True,
        "n_tags": "#tag_vocab.len",
        "capitalization_dim": "#capitalization.dim",
        "char_emb_dim": "#embeddings_char.dim",
        "save_path": "{MODELS_PATH}/onthology_objects/model_no_pos_3_ft",
        "load_path": "{MODELS_PATH}/onthology_objects/model_no_pos_3_ft",
        "char_emb_mat": "#embeddings_char.emb_mat",
        "two_dense_on_top": True,
        "use_crf": True,
        "use_batch_norm": True,
        "embeddings_dropout": True,
        "top_dropout": True,
        "intra_layer_dropout": True,
        "l2_reg": 0,
        "learning_rate": 1e-2,
        "dropout_keep_prob": 0.5
      },
      {
        "ref": "tag_vocab",
        "in": ["y_predicted"],
        "out": ["tags"]
      }
    ],
    "out": ["x_tokens", "tags"]
  },
    "train": {
        "epochs": 100,
        "batch_size": 64,
        "metrics": [
            {
                "name": "ner_f1",
                "inputs": [
                    "y",
                    "tags"
                ]
            }
        ],
        "validation_patience": 7,
        "val_every_n_epochs": 1,
        "log_every_n_epochs": -1,
        "show_examples": False,
        "validate_best": True,
        "test_best": True
    },
    "metadata": {
        "variables": {
            "ROOT_PATH": "~/.deeppavlov",
            "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
            "MODELS_PATH": "{ROOT_PATH}/models"
        },
        "requirements": [
            "{DEEPPAVLOV_PATH}/requirements/fasttext.txt",
            "{DEEPPAVLOV_PATH}/requirements/tf.txt"
        ],
        "labels": {
            "telegram_utils": "NERCoNLL2003Model",
            "server_utils": "NER"
        },
        "download": [
            {
                "url": "http://files.deeppavlov.ai/deeppavlov_data/embeddings/wiki.en.bin",
                "subdir": "{DOWNLOADS_PATH}/embeddings"
            }
        ]
    }
}

In [0]:
import json

with open("config.json", "w") as fout:
  json.dump(config, fout, ensure_ascii=False, indent=2)

In [0]:
model2 = train_model(config)

2019-12-18 16:54:11.994 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
2019-12-18 16:54:12.279 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/word_3_ft.dict]
2019-12-18 16:54:12.304 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/tag_3_ft.dict]
2019-12-18 16:54:12.427 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/onthology_objects/char_3_ft.dict]
2019-12-18 16:54:12.429 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 16:54:24.440 INFO in 'deeppavlov.core.layers.tf_layers'['tf_

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 1.4864}, "time_spent": "0:00:02", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 7}}


2019-12-18 16:54:40.675 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1032 phrases; found: 112 phrases; correct: 0.

precision:  33.04%; recall:  3.59%; FB1:  6.47

	O: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	artifact: precision:  35.71%; recall:  3.05%; F1:  5.62 14

	cognition: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	event: precision:  32.65%; recall:  13.01%; F1:  18.60 98

	geological_formation: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	human_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	living_thing: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	location: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	motivation: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	natural_object: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	natural_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	organic_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	property: precision:  0.00%; recall:  0.0

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 6.4685}, "time_spent": "0:00:15", "epochs_done": 1, "batches_seen": 28, "train_examples_seen": 1770, "impatience": 0, "patience_limit": 7}}


2019-12-18 16:54:52.676 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1032 phrases; found: 622 phrases; correct: 0.

precision:  37.46%; recall:  22.58%; FB1:  28.17

	O: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	artifact: precision:  42.70%; recall:  23.17%; F1:  30.04 89

	cognition: precision:  33.93%; recall:  11.66%; F1:  17.35 56

	event: precision:  35.64%; recall:  67.07%; F1:  46.54 463

	geological_formation: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	human_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	living_thing: precision:  78.57%; recall:  8.80%; F1:  15.83 14

	location: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	motivation: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	natural_object: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	natural_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	organic_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	property: precision:  0.00%; 

...

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 58.8294}, "time_spent": "0:00:02"}}


2019-12-18 17:00:52.446 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 4290 tokens with 973 phrases; found: 940 phrases; correct: 0.

precision:  60.00%; recall:  57.97%; FB1:  58.96

	O: precision:  60.00%; recall:  35.29%; F1:  44.44 20

	artifact: precision:  70.59%; recall:  57.14%; F1:  63.16 119

	cognition: precision:  58.25%; recall:  61.08%; F1:  59.63 194

	economic_process: precision:  9.09%; recall:  25.00%; F1:  13.33 11

	event: precision:  58.65%; recall:  58.65%; F1:  58.65 266

	geological_formation: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	human_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	land: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	living_thing: precision:  79.49%; recall:  73.81%; F1:  76.54 117

	location: precision:  61.90%; recall:  50.00%; F1:  55.32 21

	motivation: precision:  66.67%; recall:  100.00%; F1:  80.00 3

	natural_object: precision:  33.33%; recall:  30.00%; F1:  31.58 9

	natural_process: p

{"test": {"eval_examples_count": 222, "metrics": {"ner_f1": 58.965}, "time_spent": "0:00:01"}}


2019-12-18 17:00:52.891 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/word_3_ft.dict]
2019-12-18 17:00:52.901 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/tag_3_ft.dict]
2019-12-18 17:00:52.902 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/char_3_ft.dict]
2019-12-18 17:00:52.904 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 17:01:03.675 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with 

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/onthology_objects/model_no_pos_3_ft


In [0]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

config["train"]["show_examples"] = True
model2 = train_evaluate_model_from_config(config, to_train=False)

2019-12-18 17:01:05.758 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
2019-12-18 17:01:05.970 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/word_3_ft.dict]
2019-12-18 17:01:05.980 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/tag_3_ft.dict]
2019-12-18 17:01:05.981 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/onthology_objects/char_3_ft.dict]
2019-12-18 17:01:05.983 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 67: [loading fastText embeddings from `/root/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2019-12-18 17:01:17.332 INFO in 'deeppavlov.core.layers.tf_lay

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/onthology_objects/model_no_pos_3_ft


2019-12-18 17:01:20.429 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 5037 tokens with 1032 phrases; found: 984 phrases; correct: 0.

precision:  60.26%; recall:  57.46%; FB1:  58.83

	O: precision:  100.00%; recall:  100.00%; F1:  100.00 20

	artifact: precision:  79.80%; recall:  48.17%; F1:  60.08 99

	cognition: precision:  61.44%; recall:  57.67%; F1:  59.49 153

	economic_process: precision:  0.00%; recall:  0.00%; F1:  0.00 2

	event: precision:  53.18%; recall:  64.63%; F1:  58.35 299

	geological_formation: precision:  72.22%; recall:  68.42%; F1:  70.27 18

	human_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	living_thing: precision:  83.16%; recall:  63.20%; F1:  71.82 95

	location: precision:  52.50%; recall:  60.00%; F1:  56.00 40

	motivation: precision:  100.00%; recall:  100.00%; F1:  100.00 3

	natural_object: precision:  37.50%; recall:  33.33%; F1:  35.29 16

	natural_process: precision:  21.05%; recall:  33.33%; F1:  25.81 19

{"valid": {"eval_examples_count": 217, "metrics": {"ner_f1": 58.8294}, "time_spent": "0:00:02", "examples": [{"x": ["the", "key", "consideration", "is", "whether", "and", "to", "what", "extent", "the", "administrative", "record", "data", "re", "examined", "retroactively", "a", "decade", "later", "for", "the", "original", "study's", "time", "period", "would", "yield", "comparable", "results", "to", "those", "based", "on", "data", "acquired", "at", "the", "time", "of", "the", "study", "."], "y_predicted": [["O", "B-property", "B-cognition", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-time_period", "O", "O", "O", "B-artifact", "O", "B-property", "B-time", "O", "B-event", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-property", "O", "O", "B-living_thing", "O"], ["the", "key", "consideration", "is", "whether", "and", "to", "what", "extent", "the", "administrative", "record", "data", "re", "examined", "retroactively", "a", "decade", "later", "for", "the", 

2019-12-18 17:01:21.468 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 4290 tokens with 973 phrases; found: 940 phrases; correct: 0.

precision:  60.00%; recall:  57.97%; FB1:  58.96

	O: precision:  60.00%; recall:  35.29%; F1:  44.44 20

	artifact: precision:  70.59%; recall:  57.14%; F1:  63.16 119

	cognition: precision:  58.25%; recall:  61.08%; F1:  59.63 194

	economic_process: precision:  9.09%; recall:  25.00%; F1:  13.33 11

	event: precision:  58.65%; recall:  58.65%; F1:  58.65 266

	geological_formation: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	human_process: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	land: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	living_thing: precision:  79.49%; recall:  73.81%; F1:  76.54 117

	location: precision:  61.90%; recall:  50.00%; F1:  55.32 21

	motivation: precision:  66.67%; recall:  100.00%; F1:  80.00 3

	natural_object: precision:  33.33%; recall:  30.00%; F1:  31.58 9

	natural_process: p

{"test": {"eval_examples_count": 222, "metrics": {"ner_f1": 58.965}, "time_spent": "0:00:02", "examples": [{"x": ["ber", "ep4", "and", "cd44v6", "were", "shown", "to", "be", "great", "markers", "for", "detecting", "lnmm", "."], "y_predicted": [["O", "O", "O", "O", "O", "O", "O", "O", "B-living_thing", "O", "O", "O", "O", "O"], ["ber", "ep4", "and", "cd44v6", "were", "shown", "to", "be", "great", "markers", "for", "detecting", "lnmm", "."]], "y_true": ["O", "O", "O", "O", "O", "O", "O", "O", "B-living_thing", "O", "O", "B-event", "O", "O"]}, {"x": ["aims", "human", "papillomavirus", "hpv", "is", "known", "as", "causative", "for", "squamous", "cell", "carcinoma", "scc", "of", "the", "oropharynx", ",", "but", "is", "also", "found", "not", "infrequently", "in", "carcinomas", "of", "the", "sinonasal", "tract", "."], "y_predicted": [["O", "B-living_thing", "O", "O", "O", "O", "O", "O", "O", "B-O", "I-O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-